In [1]:
# shopee.py
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.chrome.options import Options
import time

# Scraping Link Produk

## Fungsi Scraping Link Produk
Fungsi untuk scraping link produk yang sesuai dengan kata kunci

In [2]:

def scrapeLink(list_product, category):
    chrome_options = Options()
    # chrome_options.add_argument('--headless')
    chrome_options.add_argument('log-level=2')
    driver = webdriver.Chrome('chromedriver', options=chrome_options)
    # katakunci = input('Masukkan kata kunci : ')

    def search(katakunci):
        links = []
        print('mencari semua product dengan kata kunci ' + katakunci)
        url = 'https://shopee.co.id/search?keyword=' + katakunci
        try:
            driver.get(url)
            time.sleep(5)
            driver.execute_script('window.scrollTo(0, 1500);')
            time.sleep(5)
            driver.execute_script('window.scrollTo(0, 2500);')
            time.sleep(5)
            soup_a = BeautifulSoup(driver.page_source, 'html.parser')
            products = soup_a.find('div', class_='row shopee-search-item-result__items')
            for link in products.find_all('a'):
                links.append('https://shopee.co.id/' + str(link.get('href')))
                # print(link.get('href'))
        except TimeoutException:
            print('failed to get links with query ' )
        return links
    
    for katakunci in list_product:
        print('Scrape link ' + katakunci)
        product_urls = search(katakunci)
        dict_urls = {'url' : product_urls}
        df = pd.DataFrame(data = dict_urls)
        PATH = '../data/raw/'+ category +'/link_product'
        file_name = PATH + '/links_' + katakunci +'.csv'
        df.to_csv(file_name, index=False)


## Kategori dan Kata Kunci
Menginisiasi list kategori dan kata kunci

In [3]:
CATEGORY = ['fashion', 'food', 'electronic']

fashion = ['hoodie', 'hijab', 'celana', 'kemeja', 'gamis', 'sepatu', 'kaos', 'sandal', 'tas', 'jeans', 'jaket', 'sweater']
food = ['snack', 'daging', 'selai', 'gula', 'susu', 'mie', 'roti', 'bumbu', 'camilan', 'bahan_kue', 'minuman', 'bahan_pokok']
electronic = ['laptop', 'mouse', 'handphone', 'earphone', 'tv', 'mixer', 'setrika', 'mesin_cuci', 'penyedot_debu', 'kamera', 'speaker', 'lampu']


## Scraping Link Produk
Scraping link produk berdasarkan kata kunci

In [4]:
# for category in [fashion, food, electronic]:
#     scrapeLink(category)

# Scraping Ulasan Produk
Scraping ulasan produk dari link yang telah ada

In [5]:
from os import walk
import re
import timeit
import requests

## Fungsi Scraping Ulasan

In [6]:
def shopeeScraper (url):
    url = url
    r = re.search(r'i\.(\d+)\.(\d+)', url)
    shop_id, item_id = r[1], r[2]
    ratings_url = 'https://shopee.co.id/api/v2/item/get_ratings?filter=0&flag=1&itemid={item_id}&limit=20&offset={offset}&shopid={shop_id}&type=0'
    data_scrape = []
    start = timeit.default_timer()
    runtime = 0
    offset = 0
    print('Scraping Process...')
    while True:   
        data = requests.get(ratings_url.format(shop_id=shop_id, item_id=item_id, offset=offset)).json()

        i = 1
        try :
            for i, rating in enumerate(data['data']['ratings'], 1):
                if rating['comment'] == '':
                    pass
                else:
                    data_scrape.append([rating['rating_star'], rating['comment']])
        except :
            pass

        if i % 20:
            break

        stop = timeit.default_timer()
        runtime+= (stop-start)
        print(runtime)
        offset += 20
    print('Scraping Done.')
    stop = timeit.default_timer()
    print('Time: ', stop - start) 
    df = pd.DataFrame(data_scrape, columns=['rating', 'reviews'])
    df = df.dropna(axis=0)
    return df

## List File Link
Membuat list file dari direktori link_product

In [7]:
LINK_PRODUCT = {'fashion' : [], 'food' : [], 'electronic' : []}

for ctg in CATEGORY:
    for (dirpath, dirnames, filenames) in walk('../data/raw/link_product/' + ctg):
        LINK_PRODUCT[ctg].extend(filenames)
        break


## Scraping Ulasan Produk
Scraping Ulasan Produk dari daftar link produk di folder link_product

In [8]:
dataReviews = pd.DataFrame({'rating' : [], 'reviews' : []})
ctg = 'fashion'
file = LINK_PRODUCT[ctg][11]
links = pd.read_csv('../data/raw/link_product/' + ctg + '/' + file)
for i, link in enumerate(links['url']):
    if i >= 47:
        print('Scraping ' + file + ' Link ke-' + str(i))
        reviews = shopeeScraper(link)
        frames = [dataReviews, reviews]
        result = pd.concat(frames).drop_duplicates().reset_index(drop=True)
        dataReviews = result
        if i % 2 == 0:
            dataReviews.to_csv('../data/raw/reviews_product/' + ctg + '/tas_v' + str(i/2) + '.csv', index=False)


Scraping links_tas.csv Link ke-47
Scraping Process...
0.4538466999999997
1.2484729999999997
5.464593099999999
10.113384199999999
15.169870799999998
20.618527799999995
26.488127499999997
32.7632044
39.472645
46.646755
54.2133807
62.1998016
70.6199965
79.4216206
88.63012499999999
98.39797279999999
108.67848029999999
119.44447739999998
130.59028159999997
142.10124829999998
154.0181452
166.2922302
178.9135256
191.92175980000002
205.28062680000002
219.07338130000002
233.28015600000003
247.95679010000003
263.010328
278.4660389
294.3199562
310.5807235
327.2779529
344.4426753
362.0003669
379.9408123
398.2759569
416.9813388
436.0920862
455.5988048
475.588397
495.9788441
516.759609
537.9704441
559.5955706
581.7247244
604.2621953
627.1989682000001
650.5281871000001
674.3361894000001
698.5608132000001
723.1746730000001
748.2281912000001
773.6287934000001
799.4064807000001
825.5763106
852.1459968
879.0750683
906.3331323
933.9419239
968.9240931
1004.2600958
1039.9246050000002
1075.8979553000001
1112

In [14]:
FASHION_LINK_MERGE = []
ctg = 'fashion'
for (dirpath, dirnames, filenames) in walk('../data/raw/reviews_product/' + ctg):
        FASHION_LINK_MERGE.extend(filenames)
        break

merge_list, count_data = [], 0
for i in FASHION_LINK_MERGE:
    data = pd.read_csv('../data/raw/reviews_product/fashion/' + i)
    count_data += len(data)
    merge_list.append(data)
merge_result = pd.concat(merge_list).drop_duplicates().reset_index(drop=True)
merge_result = merge_result.drop('Unnamed: 0', axis=1)
merge_result.to_csv('../data/raw/fashion_reviews.csv', index=False)

print(count_data)
print(merge_result)

3844621
        rating                                            reviews
0          5.0  Sumpah baguus bangeeet doong ...  Agak kebesar...
1          5.0  Alhamdulilah paket sudah sampai\nMasya Allah b...
2          5.0  Barang sudah sampai sesuai pesanan ukuran 34 m...
3          5.0  BAGUS BANGET LOHHH, monangissss harga murce ba...
4          5.0  Assalamu'alaikum wr wb. Terkait kegiatan PKKS ...
...        ...                                                ...
627163     5.0       barangnya bagus tapi packing nya kurang aman
627164     4.0      Barang nya bagus sesuai dengan pesanan nya 😘😘
627165     5.0  tasnya sesuai gambar murah tp ngak murahan. Ng...
627166     5.0                     Sumpah sebagus ini 30k 🥲❤❤❤❤❤❤
627167     5.0                        Bagussss bangeetttt ngetttt

[627168 rows x 2 columns]
